In [1]:
import pandas as pd
import numpy as np

In [4]:
data = {'name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'age': [42, 52, 36, 24, 73], 
        'preTestScore': [4, 24, 31, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70]}
df = pd.DataFrame(data, columns=['name', 'age', 'preTestScore', 'postTestScore'])
df

,name,age,preTestScore,postTestScore
0,Jason,42,4,25
1,Molly,52,24,94
2,Tina,36,31,57
3,Jake,24,2,62
4,Amy,73,3,70


In [7]:
df['elderly'] = np.where(df['age']>=50, 1, 0)
df

,name,age,preTestScore,postTestScore,elderly
0,Jason,42,4,25,0
1,Molly,52,24,94,1
2,Tina,36,31,57,0
3,Jake,24,2,62,0
4,Amy,73,3,70,1


In [9]:
df['labels'] = [1,0,1,0,1]
df

,name,age,preTestScore,postTestScore,elderly,labels
0,Jason,42,4,25,0,1
1,Molly,52,24,94,1,0
2,Tina,36,31,57,0,1
3,Jake,24,2,62,0,0
4,Amy,73,3,70,1,1


In [57]:
numericColumns = ['age']
binaryColumns = ['elderly']
denom = {}
ar = {}
for col in numericColumns:
    denom[col] = df[col].mean();

for col in numericColumns:
    ar[col] = df.fillna(value=0.0).groupby('labels')[[col]].mean().max().values[0]/denom[col]

def test_sum(series):
    return (series.sum()/(len(series)-series.sum()))
for col in binaryColumns:
    groups = df.groupby('labels')[[col]]
    ar[col] = groups.aggregate([test_sum]).max().values[0]
ar

{'age': 1.1086637298091044, 'elderly': 1.0}